# UPenn College House Distances
This notebook determines the optimal route to take to visit all of the college houses at Penn. I made this so I can figure out the best route to hang up the posters advertising our Soundworks Tap Factory (tap dancing group at Penn) semesterly show.

In [1]:
%%capture
!pip install -q folium ortools geopy

In [2]:
import folium
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from geopy.distance import geodesic
import numpy as np

# College house locations with latitudes and longitudes
locations = [
    (39.95391067440265, -75.20259689153883),  # Gutman
    (39.952292747206734, -75.20242836759456),  # Gregory
    (39.95268180174318, -75.20010579457704),   # Harnwell
    (39.953289561917, -75.20141670503021),   # Rodin
    (39.9523259070965, -75.20104757434025),   # Harisson
    (39.9547131409648, -75.201454355957),  # Radian
    (39.95096631832606, -75.19729520317654),  # Fisher Hassenfeld
    (39.950584865913015, -75.19703205929552),  # Ware
    (39.950439391129, -75.19612121666785),  # Riepe
    (39.95114976199825, -75.19859403201278),   # Stouffer
    (39.95172158789118, -75.20013221852159),   # Stouffer Mayer
    (39.95389647973634, -75.19133088968516),   # Lauder
    (39.95338639383971, -75.19072518293949),  # Hill
    (39.95550349223309, -75.194488425267),    # Axis
    (39.954259378645396, -75.19446424550384), # Kings Court English
]

names = ["Gutman", "Gregory", "Harnwell", "Rodin", "Harisson", "Radian", "Fisher Hassenfeld", "Ware", "Riepe", "Stouffer", "Stouffer Mayer", "Lauder", "Hill", "Axis", "Kings Court English"]

# Function to calculate distance matrix using geopy (Haversine distance)
def create_distance_matrix(locations):
    size = len(locations)
    distance_matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            if i == j:
                distance_matrix[i][j] = 0
            else:
                distance_matrix[i][j] = geodesic(locations[i], locations[j]).kilometers
    return distance_matrix

# Create the distance matrix
distance_matrix = create_distance_matrix(locations)

# Solve the TSP problem using OR-Tools
def create_data_model(distance_matrix):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0  # Starting and ending point
    return data

def get_tsp_solution_route(manager, routing, solution):
    """Extracts the solution as a list of nodes in the order of the route."""
    route = []
    index = routing.Start(0)
    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))
    route.append(manager.IndexToNode(index))  # Return to the start
    return route

def solve_tsp(distance_matrix):
    """Solves the TSP and returns the solution route."""
    # Instantiate the data problem.
    data = create_data_model(distance_matrix)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(data['distance_matrix'][from_node][to_node])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Get the route from the solution.
    if solution:
        return get_tsp_solution_route(manager, routing, solution)
    else:
        return None

# Solve TSP and get the route
route = solve_tsp(distance_matrix)

# Create a folium map centered around the geographical center of the locations
avg_lat = np.mean([loc[0] for loc in locations])
avg_lon = np.mean([loc[1] for loc in locations])
tsp_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=16)

# Plot the locations as markers
for i, (lat, lon) in enumerate(locations):
    folium.Marker([lat, lon], popup=names[i]).add_to(tsp_map)

# Draw the TSP route as lines
for i in range(len(route) - 1):
    start = locations[route[i]]
    end = locations[route[i + 1]]
    folium.PolyLine([start, end], color="blue", weight=2.5, opacity=1).add_to(tsp_map)

# Save the map to an HTML file and display it
tsp_map.save("tsp_solution_map.html")
tsp_map
